In [1]:
import torch

In [2]:
torch.rand(3,2)

tensor([[0.0717, 0.0865],
        [0.8366, 0.1215],
        [0.4406, 0.4916]])

In [4]:
from transformers import AutoformerConfig, AutoformerForPrediction